# Q&A over LangChain Docs
[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-benchmarks/blob/main/docs/source/notebooks/retrieval/langchain_docs_qa.ipynb)

Let's evaluate your architecture on a Q&A dataset for the LangChain python docs. For more examples of how to test different embeddings, indexing strategies, and architectures, see the [Evaluating RAG Architectures on Benchmark Tasks](./comparing_techniques.ipynb) notebook.

## Pre-requisites

We will install quite a few prerequisites for this example since we are comparing many techniques and models.

We will be using LangSmith to capture the evaluation traces. You can make a free account at [smith.langchain.com](https://smith.langchain.com/). Once you've done so, you can make an API key and set it below.

In [ ]:
%pip install -U --quiet langchain langsmith langchainhub langchain_benchmarks
%pip install --quiet chromadb openai huggingface pandas langchain_experimental sentence_transformers pyarrow anthropic tiktoken

For this code to work, please configure LangSmith environment variables with your credentials.

In [ ]:
import os

os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "ls_..."  # Your API key

In [ ]:
# Update these with your own API keys
os.environ["ANTHROPIC_API_KEY"] = "sk-..."
os.environ["OPENAI_API_KEY"] = "sk-..."
# Silence warnings from HuggingFace
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Review Q&A Tasks

The registry provides configurations to test out common architectures on curated datasets.

In [ ]:
from langchain_benchmarks import clone_public_dataset, registry

In [ ]:
registry = registry.filter(Type="RetrievalTask")
registry

In [ ]:
langchain_docs = registry["LangChain Docs Q&A"]
langchain_docs

In [ ]:
clone_public_dataset(langchain_docs.dataset_id, dataset_name=langchain_docs.name)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-base",
    model_kwargs={"device": 0},  # Comment out to use CPU
)

docs = langchain_docs.get_docs()
retriever_factory = langchain_docs.retriever_factories["basic"]
# Indexes the documents with the specified embeddings
# Note that this does not apply any chunking to the docs,
# which means the documents can be of arbitrary length
retriever = retriever_factory(embeddings, docs=docs)

In [ ]:
from operator import itemgetter
from typing import Sequence

from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain.schema.document import Document
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign


def format_docs(docs: Sequence[Document]) -> str:
    formatted_docs = []
    for i, doc in enumerate(docs):
        doc_string = (
            f"<document index='{i}'>\n"
            f"<source>{doc.metadata.get('source')}</source>\n"
            f"<doc_content>{doc.page_content}</doc_content>\n"
            "</document>"
        )
        formatted_docs.append(doc_string)
    formatted_str = "\n".join(formatted_docs)
    return f"<documents>\n{formatted_str}\n</documents>"


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI assistant answering questions about LangChain."
            "\n{context}\n"
            "Respond solely based on the document content.",
        ),
        ("human", "{question}"),
    ]
)
llm = ChatAnthropic(model="claude-2.1", temperature=1)

response_generator = (prompt | llm | StrOutputParser()).with_config(
    run_name="GenerateResponse",
)
chain = (
    RunnableAssign(
        {
            "context": (itemgetter("question") | retriever | format_docs).with_config(
                run_name="FormatDocs"
            )
        }
    )
    | response_generator
)

In [ ]:
chain.invoke({"question": "What's expression language?"})

### Evaluate

Let's evaluate your RAG architecture on the dataset now.

In [ ]:
from langsmith.client import Client

from langchain_benchmarks.rag import get_eval_config

In [ ]:
client = Client()
RAG_EVALUATION = get_eval_config()

test_run = client.run_on_dataset(
    dataset_name=langchain_docs.name,
    llm_or_chain_factory=chain,
    evaluation=RAG_EVALUATION,
    verbose=True,
)

In [ ]:
test_run.get_aggregate_feedback()

## Evaluate with a default factory

The task can define default chain and retriever "factories", which provide a default architecture that you can modify by choosing the llms, prompts, etc. Let's try the `conversational-retrieval-qa` factory.

In [ ]:
# Factory for creating a conversational retrieval QA chain
chain_factory = langchain_docs.architecture_factories["conversational-retrieval-qa"]

In [ ]:
from langchain.chat_models import ChatAnthropic

# Example
llm = ChatAnthropic(model="claude-2", temperature=1)


chain = chain_factory(retriever, llm=llm)

chain.invoke({"question": "What is expression language?"})

In [ ]:
test_run = client.run_on_dataset(
    dataset_name=langchain_docs.name,
    llm_or_chain_factory=chain,
    evaluation=RAG_EVALUATION,
    verbose=True,
)

In [ ]:
test_run.get_aggregate_feedback()